## Estudo dirigido de Economia Regional

1.Em quais estados brasileiros estão concentrados o emprego? Comparando com os EUA, a concentração do emprego é maior ou menor (dados disponíveis na SIDRA-IBGE RAIS e na PNAD)

2.Qual a percentagem da população brasileira, em 2018 vivia nos estados que detêm a maior parcela do emprego? Fazer um gráfico mostrando como vem sendo esta evolução de 1999 ate 2019.

3.Quais e quantos são os estados brasileiros cujo PIB per capita é inferior a 75% do PIB nacional em 2019? Qual a percentagem da população brasileira vivi nestes estados em 2019?

4.Faça um gráfico do PIB per capita das 5 macro regiões brasileiras (Norte, Nordeste, Sul, Sudeste e Centro Oeste) entre 1996 a 2021. Identifique se existiu redução da desigualdade.

5.Faça um mapa do Brasil para os itens abaixo e compare os resultados:
- taxas de desemprego
- PIB industrial per capita
 

6.Faça um mapa do Brasil para, para os itens abaixo e compare os resultados:
- Índice de Gini
- renda per capita

### Importar planilhas e tratar dados

In [1]:
# bibliotecas usadas
import pandas as pd
import numpy as np
import matplotlib as plt

In [3]:
pop_residente_regiao = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de Dados\Estudo dirigido - Economia regional\populacao residente por regiao (1996-2021).csv', sep=';')
pop_empregada2018 = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de Dados\Estudo dirigido - Economia regional\populacao empregada 2018.csv', sep=';')
pop_residente_estado = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de Dados\Estudo dirigido - Economia regional\populacao residente por estado (1999-2019).csv', sep=';')
pib_per_capita = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de Dados\Estudo dirigido - Economia regional\pib estadual per capita (1996-2021).csv', sep=';')
pib_industrial2021 = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de Dados\Estudo dirigido - Economia regional\pib estadual industrial 2021.csv', sep=';')


<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\{'
<>:5: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\{'
<>:5: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_7064\3101053893.py:1: SyntaxWarning: invalid escape sequence '\{'
  pop_residente_regiao = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de Dados\Estudo dirigido - Economia regional\populacao residente por regiao (1996-2021).csv', sep=';')
C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_7064\3101053893.py:2: SyntaxWarning: invalid escape sequence '\{'
  pop_empregada2018 = pd.read_csv('M:\{estudos}\Python\Estudos-Python\Análise de

### Questões propostas

1. Em quais estados brasileiros estão concentrados o emprego? Comparando com os EUA, a concentração do emprego é maior ou menor (dados disponíveis na SIDRA-IBGE RAIS e na PNAD)

2. Qual a percentagem da população brasileira, em 2018 vivia nos estados que detêm a maior parcela do emprego? Fazer um gráfico mostrando como vem sendo esta evolução de 1999 ate 2019.



3. Quais e quantos são os estados brasileiros cujo PIB per capita é inferior a 75% do PIB nacional em 2019? Qual a percentagem da população brasileira vivi nestes estados em 2019?


4. Faça um gráfico do PIB per capita das 5 macro regiões brasileiras (Norte, Nordeste, Sul, Sudeste e Centro Oeste) entre 1996 a 2021. Identifique se existiu redução da desigualdade.

5. Faça um mapa do Brasil para os itens abaixo e compare os resultados:
- taxas de desemprego
- PIB industrial per capita



6. Faça um mapa do Brasil para, para os itens abaixo e compare os resultados:
- Índice de Gini
- renda per capita